---
# Computer Vision and Imaging Extended [30241] - Assessed Assignent
---

## Part 1 

### Imports and Load data 

In [62]:
# General Imports
from scipy.io import loadmat 
import numpy as np

# Display Import 
from ipywidgets import interact, fixed
import pandas as pd

# K - Mean Clustering 
from sklearn.cluster import KMeans

# Hungarian Mathching 
from scipy.optimize import linear_sum_assignment

# Otsu segmentation 
from skimage.filters import threshold_multiotsu


# Thresholding
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from matplotlib import pyplot as plt

In [63]:
# Load .mat files in python: 
data = loadmat('Brain-1.mat')
T1 = data['T1']
label = data['label']

# extract Number of Slices 
num_slices = T1.shape[2]
# Number of classses 
num_classes = 6

print(data.keys())
print(T1.shape)
print(label.shape)



dict_keys(['__header__', '__version__', '__globals__', 'T1', 'label'])
(362, 434, 10)
(362, 434, 10)


----
### Evaluation Methods/Function & Display image function
Evaluation method are used to evaluate method 1,2,3. 2 Main evaluation are uses
1. IOU
2. Dice

The Display Image Function display images after running method 1,2,3. It displayed image
1. Original Slice
2. Ground Truth Slice
3. Segmented Label Slice 

In [64]:
# Evaluation Method 1 IOU 
def iou_score(prediction, img_gt):
    ious = []
    for c in range(num_classes):
        intersection = np.sum((prediction == c) & (img_gt == c))
        union = np.sum((prediction == c) | (img_gt == c))
        ious.append(intersection / union if union > 0 else 0)
    return ious

# Evaluation Method 2 Dice 
def dice_score(prediction, img_gt):
    dice = []
    for c in range(num_classes):
        intersection = 2 * np.sum((prediction == c) & (img_gt == c))
        total = np.sum(prediction == c) + np.sum(img_gt == c)
        dice.append(intersection / total if total > 0 else 0)
    return dice

# Display Slice Function 
def show_slice(i, method, name):
    
    plt.figure(figsize=(15,5))

    # add main title 
    plt.suptitle(f"{name} Segmentation – Slice {i}", fontsize=14)
    
    # subplot 1 show original slices
    plt.subplot(1,3,1)
    plt.imshow(T1[:,:,i], cmap='gray')
    plt.title(f"Original Slice")
    plt.axis('off')
    
    # subplot 2 show ground truth slice 
    plt.subplot(1,3,2)
    plt.imshow(label[:,:,i], cmap='jet')
    plt.title(f"Ground Truth Slice")
    plt.axis('off')
    
    # subplot 3 show predicted_maks based on method 
    plt.subplot(1,3,3)
    plt.imshow(method[:,:,i], cmap='jet')
    plt.title(name)
    plt.axis('off')
    
    plt.show()
    


### Preprocess Image and Hungarian match and map Function 
- Preproecssing the image by normalising the image and applying a gaussian filter as it help to filter out noise
- This function are used after segmentation function to map the predicted masks to ground truth 

In [65]:
# def intensity_sort_mapping(volume, pred_labels, num_classes=6):
#     # Compute mean intensity per cluster
#     means = [np.mean(volume[pred_labels == c]) for c in range(num_classes)]

#     # Sort clusters by intensity
#     sorted_clusters = np.argsort(means)

#     # Map sorted clusters to GT classes 0..num_classes-1
#     mapping = {sorted_clusters[i]: i for i in range(num_classes)}
#     return mapping
# Normalisation and filtering slices/img  

def pre_process(img): 
    img_norm = (img - img.min()) / (img.max() - img.min())
    # 2. Smooth
    img_filter = gaussian_filter(img_norm, sigma=2)
    return img_filter
    
def hungarian_match(prediction, ground_truth):
    # Do overlap matrix 
    overlap = np.zeros((num_classes, num_classes))
    for i in range(num_classes): 
        for j in range(num_classes):
            overlap[i,j] = np.sum((prediction == i) & (ground_truth == j))

    # Try to maximise overlap 
    cost = -overlap 
    
    # predicted cluster, and matched ground truth class 
    rows, cols = linear_sum_assignment(cost)
    mapping = dict(zip(rows, cols))
    return mapping 

# Uses hungarian_match to apply mapping to predicted mask 
def apply_label_map(prediction, mapping): 
    mapped = np.zeros_like(prediction) 
    for pred_label, gt_label in mapping.items():
        mapped[pred_label == prediction] = gt_label
    return mapped
  

### Method 1: K-Mean Clustering

In [66]:
# Cluster Function 
# set cluster group (6 because there is 6 label including air) 
def cluster_slices(img, k=6):
    
    # reshape to (N_pixels, 1) 
    h, w = img.shape 
    data = img.reshape((-1,1))

    # Initialise the KMeans object with number of clusters
    kmeans = KMeans(n_clusters=k, random_state=0)

    # Fit the dat and kmeans mode
    kmeans.fit(data) 

    #Obtain the prediction of all data points of the reshaped image 
    labels = kmeans.predict(data).reshape(h,w)
    return labels 


In [67]:
# Create array to store the predicted masks (to calculate with ground truth) and segmented_images (to display) 
kmean_predicted_masks = np.zeros_like(label)
# segmented_images = np.zeros_like(T1, dtype=np.uint8) 

# Apply K-Mean clustering Segmentation Method
for i in range(num_slices):

    # Extract img and label slices
    img = T1[:,:,i]
    img_gt = label[:,:,i]

    # preprocess before running K-Mean 
    img_filter = pre_process(img) 
    # Run K mean on i image 
    prediction = cluster_slices(img_filter)
    
    # Compute Hungarian Mapping 
    mapping = hungarian_match(prediction, img_gt)
    
    # Apply Mapping
    pred_mapped = apply_label_map(prediction, mapping)

    # store result
    kmean_predicted_masks[:,:,i] = pred_mapped 
     
 

### Evaluation for K-Mean Clustering 

In [68]:
# Apply IOU, Dice

kmean_iou_all = np.zeros((num_slices, num_classes))
kmean_dice_all = np.zeros((num_slices, num_classes)) 

print("Evaluating K-Mean-Clustering Segmentation Per slice")
for i in range(num_slices): 
    kmean_pred_slice = kmean_predicted_masks[:,:,i]
    img_gt = label[:,:,i]

    # compute kmean iou and store score for each slices 
    kmean_iou = iou_score(kmean_pred_slice, img_gt)
    kmean_dice = dice_score(kmean_pred_slice, img_gt)   
    
    # Store all iou and dice score to compute mean later
    kmean_iou_all[i] = kmean_iou
    kmean_dice_all[i] = kmean_dice 

    # print iou and dice score slices by slice with 3 decimal places
    print(f" Slice {i} IoU score: {[f'{x:.3f}' for x in kmean_iou]}")
    print(f" Slice {i} IoU score: {[f'{x:.3f}' for x in kmean_iou]}")


# calculate the kmean average iou score
kmean_iou_mean = kmean_iou_all.mean(axis=0) 
print(f"Average K-Mean Dice score: {kmean_iou_mean}")

# calculate the kmean average dice score
kmean_dice_mean = kmean_dice_all.mean(axis=0) 
print(f"Average K-Mean Dice score: {kmean_dice_mean}") 

interact(show_slice, i=(0, num_slices-1), method=fixed(kmean_predicted_masks), name=fixed("K-Means"))


Evaluating K-Mean-Clustering Per slice
 Slice 0 IoU score: ['0.892', '0.064', '0.382', '0.347', '0.465', '0.576']
 Slice 0 IoU score: ['0.892', '0.064', '0.382', '0.347', '0.465', '0.576']
 Slice 1 IoU score: ['0.882', '0.061', '0.294', '0.284', '0.497', '0.530']
 Slice 1 IoU score: ['0.882', '0.061', '0.294', '0.284', '0.497', '0.530']
 Slice 2 IoU score: ['0.890', '0.065', '0.354', '0.338', '0.467', '0.573']
 Slice 2 IoU score: ['0.890', '0.065', '0.354', '0.338', '0.467', '0.573']
 Slice 3 IoU score: ['0.898', '0.072', '0.384', '0.349', '0.466', '0.605']
 Slice 3 IoU score: ['0.898', '0.072', '0.384', '0.349', '0.466', '0.605']
 Slice 4 IoU score: ['0.888', '0.064', '0.306', '0.282', '0.531', '0.525']
 Slice 4 IoU score: ['0.888', '0.064', '0.306', '0.282', '0.531', '0.525']
 Slice 5 IoU score: ['0.897', '0.075', '0.384', '0.346', '0.464', '0.637']
 Slice 5 IoU score: ['0.897', '0.075', '0.384', '0.346', '0.464', '0.637']
 Slice 6 IoU score: ['0.893', '0.073', '0.381', '0.366', '0.4

interactive(children=(IntSlider(value=4, description='i', max=9), Output()), _dom_classes=('widget-interact',)…

<function __main__.show_slice(i, method, name)>

### Method 2: Otsu

In [69]:
def otsu_segment_slice(img):
    thresholds = threshold_multiotsu(img, classes=num_classes, nbins=256)

    # Digitise pixels into class labels
    segmented = np.digitize(img, bins=thresholds)
    return segmented



In [ ]:
# extract Number of Slices 
otsu_predicted_masks= np.zeros_like(label) 

for i in range(num_slices):

    # extact img and label slice
    img = T1[:, :, i]
    img_gt  = label[:, :, i]

    # 1. Preprocess slice
    img_filter = pre_process(img)

    # 2. Apply Multi-Otsu segmentation
    prediction = otsu_segment_slice(img_filter)

    # 3. Hungarian align labels
    mapping = hungarian_match(prediction, img_gt)

    # 4. Apply the mapping
    pred_mapped = apply_label_map(prediction, mapping)

    # 5. Store final mapped slice
    otsu_predicted_masks[:, :, i] = pred_mapped



### Evaluation for Otsu


In [ ]:
# Apply IOU, Dice

otsu_iou_all = np.zeros((num_slices, num_classes))
otsu_dice_all = np.zeros((num_slices, num_classes)) 

print("Evaluating Otsu Segmentation Per slice")
for i in range(num_slices): 
    otsu_pred_slice = otsu_predicted_masks[:,:,i]
    img_gt = label[:,:,i]

    # compute otsu iou and store score for each slices 
    otsu_iou = iou_score(otsu_pred_slice, img_gt)
    otsu_dice = dice_score(otsu_pred_slice, img_gt)   
    
    # Store all iou and dice score to compute mean later
    otsu_iou_all[i] = otsu_iou
    otsu_dice_all[i] = otsu_dice 

    # print iou and dice score slices by slice with 3 decimal places
    print(f" Slice {i} IoU score: {[f'{x:.3f}' for x in otsu_iou]}")
    print(f" Slice {i} IoU score: {[f'{x:.3f}' for x in otsu_iou]}")


# calculate the otsu average iou score
otsu_mean_iou_mean = otsu_iou_all.mean(axis=0) 
print(f"Average K-Mean Dice score: {otsu_iou_mean}")

# calculate the otsu average dice score
otsu_dice_mean = otsu_dice_all.mean(axis=0) 
print(f"Average K-Mean Dice score: {otsu_dice_mean}") 

interact(show_slice, i=(0, num_slices-1), method=fixed(otsu_predicted_masks), name=fixed("Otsu"))


### Method 3:  

In [11]:
from scipy.ndimage import gaussian_filter1d
import numpy as np

threshold_predicted_masks = np.zeros_like(label) 
def multi_threshold_segmentation(img, num_classes=6):
    # Calculate the raw histogram (count) and the normalised bins (*0 to 255" )
    hist, bins = np.histogram(img.flatten(), bins=np.max(img), range=(0, np.max))
    



In [12]:
from scipy.ndimage import gaussian_filter1d
import numpy as np

def multi_peak_threshold(img, num_classes=6, sigma=2.0):
    max_val = int(np.max(img))
    hist, bins = np.histogram(img.flatten(), bins=max_val, range=(0, max_val))
    hist_smooth = gaussian_filter1d(hist, sigma=sigma)

    # peak detection
    peaks = (hist_smooth[1:-1] > hist_smooth[:-2]) & (hist_smooth[1:-1] > hist_smooth[2:])
    peak_indices = np.where(peaks)[0] + 1  

    # keep strongest peaks if too many
    if len(peak_indices) > num_classes:
        peak_heights = hist_smooth[peak_indices]
        top_k = np.argsort(peak_heights)[-num_classes:]
        peak_indices = peak_indices[top_k]

    # fallback if too few peaks
    if len(peak_indices) < num_classes:
        peak_indices = np.linspace(0, max_val - 1, num_classes).astype(int)

    # sort peaks
    peak_indices = np.sort(peak_indices)

    # compute thresholds (midpoints)
    thresholds = 0.5 * (peak_indices[:-1] + peak_indices[1:])

    segmented = np.digitize(img, bins=thresholds)

    return segmented, thresholds, peak_indices, hist, hist_smooth, bins


## Part 2: Evaluation 

In [ ]:
# Run the evaluation to display mean and 

## Part 3: 3D K-Mean Clustering & Evaluation 


In [19]:

# --- Preprocessing ---
def pre_process_3d(volume):
    vol_norm = (volume - volume.min()) / (volume.max() - volume.min())
    vol_smooth = gaussian_filter(vol_norm, sigma=2)  # 3D smoothing
    return vol_smooth


# --- 3D K-Means ---
def kmeans_3d_segmentation(volume):
    H, W, S = volume.shape
    
    # Flatten into (N_voxels, 1)
    data = volume.reshape(-1, 1)
    
    # K-Means on FULL 3D volume
    kmeans = KMeans(n_clusters=num_classes, random_state=0)
    kmeans.fit(data)
    
    labels = kmeans.predict(data)
    
    # Reshape back to (H, W, S)
    labels_3d = labels.reshape(H, W, S)
    return labels_3d


# --- Hungarian Matching on full 3D volume ---
def hungarian_match_3d(prediction, img_gt):
    overlap = np.zeros((num_classes, num_classes))

    for i in range(num_classes):
        for j in range(num_classes):
            overlap[i, j] = np.sum((prediction == i) & (img_gt == j))

    cost = -overlap
    rows, cols = linear_sum_assignment(cost)
    mapping = dict(zip(rows, cols))
    return mapping, overlap


# --- Apply mapping ---
def apply_label_map_3d(prediction, mapping):
    mapped = np.zeros_like(prediction)
    for pred_label, gt_label in mapping.items():
        mapped[prediction == pred_label] = gt_label
    return mapped


# =====================================================
#               RUN FULL 3D K-MEANS PIPELINE
# =====================================================

# 1. Preprocess entire volume
T1_filt = pre_process_3d(T1)

# 2. 3D K-means segmentation
pred_3d = kmeans_3d_segmentation(T1_filt)

# 3. Hungarian mapping with GT volume
mapping_3d, overlap_3d = hungarian_match_3d(pred_3d, label)

# 4. Apply mapping
kmean_predicted_masks_3d = apply_label_map_3d(pred_3d, mapping_3d)

In [20]:
ious_3d = iou_score(kmean_predicted_masks_3d, label)
dice_3d = dice_score(kmean_predicted_masks_3d, label)

print("3D K-Means IoU:", ious_3d)
print("3D K-Means Dice:", dice_3d)

3D K-Means IoU: [0.9001164442686042, 0.06885890501187601, 0.42504365890127954, 0.3257446901304025, 0.4651652895115786, 0.5526273998478808]
3D K-Means Dice: [0.9474329291593268, 0.12884564031603576, 0.5965342272095603, 0.49141390880979013, 0.6349662974429925, 0.7118609395944251]
